In [ ]:
#@title Answer dashboard
from pathlib import Path
from ipywidgets import *
from IPython.display import clear_output
from IPython import display
import pandas as pd
import os
from os import path
import random
import io
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from datetime import datetime
import threading
import seaborn as sns
import json
import re
from QuestionsList import QuestionsList

currentQuiz= None
txtprogrs = None

Qtitle = "Question 1:"
QText = "What is a correct syntax to output \"Hello World\" in Python?"
BOLD = '\033[1m'
RESET = '\033[0m'



def read_quizes():

    global Qzss

    qzlist = []

    rel_path = 'Quizzes'
    abs_file_path = os.path.join(Path.cwd(), rel_path)
    for root, dirs, files in os.walk(abs_file_path):
        for file in files:
            if (file.find('.csv')>-1):
                qzlist.append(file[:-4])


    Qzss.options = [x for x in Qzss.options]+qzlist

    return


class Question():
    def __init__(self, title, text,mchoice, correctness):
        self.title = title
        self.text = text
        self.type = mchoice
        self.choices = []
        self.tests = []
        self.keywords = []
        self.correctchoice = None
        self.correctness = correctness

    def getTitle(self):
        return self.title

    def getText(self):
        return self.text

    def getChoices(self):
        return self.choices

    def getCorrectChoice(self):
        return self.correctchoice

    def setChoices(self,myit):
        self.choices = myit
        return

    def setTests(self,tests):
        self.tests = tests
        return

    def setKeywords(self,keywords):
        self.keywords = keywords
        return

    def IsMChoice(self):
        return self.type == 'multiple_choice'

    def isProgrammingQuestion(self):
        return self.type == 'programming'

    def isCustomQuestion(self):
        return self.type == 'custom'

    def isOpenQuestion(self):
        return self.type == 'open'

class Quiz():
    def __init__(self, name):
        self.name = name
        self.questions = []
        self.currentQuestion = None

    def getName(self):
        return self.name

    def getQuestions(self):
        return self.questions

    def setCurrentQuestion(self,myqst):
        self.currentQuestion = myqst
        return

    def getCurrentQuestion(self):
        return self.currentQuestion



class VisualManager():
    def __init__(self, questions):
        self.download_status = widgets.Label(value="Download status: Not synced yet")
        online_version = False
        from Quiz import Quiz
        from VisualManager import VisualManager
        
        if online_version:
            !git clone https://github.com/Tim-Gorter/quiz >/dev/null 2>&1
            %cd quiz
            clear_output()
            from GoogleDrive import GoogleDrive
            from google.colab import auth
            auth.authenticate_user()
            self.drive = GoogleDrive(self)
            self.drive_path = '/content/drive'
        else:
            from LocalDrive import GoogleDrive
            self.drive = GoogleDrive()
            self.drive_path = './drive'
        self.Qname= widgets.Label(value="Questions")
        self.Qqsts= widgets.Select(description="")
        self.answers_label = widgets.Label(value="Users:")
        self.answers = widgets.Select()
        self.answers.observe(self.show_selected_answer, names='value')
        self.answer_bar_chart = widgets.Output()
        self.Qqsts.layout.width ='95%'
        self.Qqsts.observe(self.open_question)
        self.description_out = widgets.Output()
        # self.answer = widgets.Output()
        self.answer_label = widgets.Label(value="Answer:")
        self.answer = widgets.Textarea(layout={'height': '150px'})
        self.correctness_label = widgets.Label(value="Correctness:")
        self.correctness = Label(value="")
        # self.keys_label = widgets.Label(value= "Keys")
        # self.keys = widgets.Select()
        # self.keys.observe(self.keys_on_change, names='value')
        self.qans_lbl= widgets.Label(value="Answer")
        self.qans_lbl.layout.visibility = 'hidden'
        self.qans_lbl.layout.display = 'none'
        self.writtenresp = widgets.Textarea(value='')
        self.writtenresp.layout.visibility = 'hidden'
        self.writtenresp.layout.display = 'none'
        self.label_test = widgets.Label(value="Test")
        self.sync_button = widgets.Button(description="Sync")
        self.sync_button.on_click(self.sync)
        self.questions = questions
        self.progress_graph = widgets.Output()
        self.progress_graph2 = widgets.Output()
        self.ML_DashboardProgressGraph = widgets.Output()
        self.progress_bar_chart = widgets.Output()

        answer_view = VBox([self.answers_label
                            ,self.answers
                            # ,self.answer_label,self.answer,self.correctness_label,self.correctness])
                            ,HBox([
                                VBox([self.answer_label,self.answer]), ])])#VBox([self.keys_label,self.keys])
        left_side = VBox([self.description_out,self.qans_lbl,answer_view], layout=Layout(width = '50%'))
        right_side = VBox([self.answer_bar_chart])

        #quiz tab
        hboxleft = VBox([self.Qname,self.Qqsts],layout=Layout(width = '15%'))
        qvbox = HBox([left_side, right_side])

        hboxmiddle = VBox(children=[qvbox,HBox(children=[self.download_status, self.sync_button]),],layout=Layout(width = '75%'))
        hboxright = HBox(children=[VBox(children=[])],layout=Layout(width = '15%'))

        self.QuizTab = HBox([hboxleft,hboxmiddle,hboxright])

        #progress tab
        with self.progress_graph:
          clear_output(wait=True)
          graph = self.get_progress_graph(questions)
          graph.show()

        with self.progress_bar_chart:
          clear_output(wait=True)
          graph = self.get_progress_bar_chart(questions)
          graph.show()

        with self.progress_graph2:
          clear_output(wait=True)
          graph = self.get_progress_graph2(questions)
          graph.show()

        with self.ML_DashboardProgressGraph:
          clear_output(wait=True)
          data = self.get_ML_DashboardProgressGraphData()
          graph = self.get_ML_DashboardProgressGraph(data)
          graph.show()

        progress_hbox_left = VBox([HBox([self.progress_graph2]),
                                   HBox([self.progress_bar_chart]),
                                   HBox([self.progress_graph]),
                                   HBox([self.download_status, self.sync_button])])

        
        self.ProgressTab = HBox([progress_hbox_left])
        self.MLDashboardTab = HBox([self.ML_DashboardProgressGraph])

    def plot_key_answers(self, key_name):
        with self.answer_bar_chart:
            clear_output(wait=True)
            values = []

            folders = os.listdir(self.drive_path)
            for user in folders:
                for question in os.listdir(os.path.join("drive", user)):
                    if question.endswith(".json"):
                        try:
                            with open(os.path.join("drive", user, question), "r") as f:
                                data = json.load(f)
                                if "answer" in data and isinstance(data["answer"], dict):
                                    if key_name in data["answer"]:
                                        values.append(data["answer"][key_name])
                        except Exception as e:
                            print(f"Error reading {question}: {e}")

            if not values:
                print(f"No values found for key '{key_name}'")
                return

            numeric_values = []
            non_numeric_values = []
            for v in values:
                try:
                    numeric_values.append(float(v))
                except (ValueError, TypeError):
                    non_numeric_values.append(str(v))

            plt.figure(figsize=(8,4))

            if len(numeric_values) == len(values):
                plt.hist(numeric_values, bins=10, color="blue", alpha=0.7)
                plt.xlabel(key_name)
                plt.ylabel("Frequency")
                plt.title(f"Distribution of numeric values for '{key_name}'")

            else:
                value_counts = pd.Series(non_numeric_values).value_counts()
                plt.bar(value_counts.index, value_counts.values, color="green")
                plt.xlabel(key_name)
                plt.ylabel("Count")
                plt.title(f"Distribution of text values for '{key_name}'")
                plt.xticks(rotation=45, ha="right")

            plt.tight_layout()
            plt.show()

    def keys_on_change(self,change):
        if not change["new"]:
            return
        selected_key = change["new"]
        self.plot_key_answers(selected_key)

    def get_progress_data(self, questions):
        question_titles = [q["title"] for q in questions]
        folders = os.listdir(self.drive_path)
        answers_dict = {}
        for user in folders:
          answers = []
          for question in os.listdir(os.path.join("drive", user)):
            if question.endswith(".json"):
              question_name = question[:-5]
              if question_name in question_titles:
                answers.append(question_name)
          answers_dict[user] = answers
        return answers_dict

          # answer_file = os.path.join(os.path.join("drive", folder), question_name + ".json")
          # if os.path.exists(answer_file):
          #     import json
          #     with open(answer_file, 'r', encoding='utf-8') as f:
          #         answer_data = json.load(f)
          #     answer = ""
          #     if answer_data["type"] == "multiple_choice":
          #         answer = answer_data["answer"]
          #     elif answer_data["type"] == "open":
          #         answer = answer_data["answer"]
          #     elif answer_data["type"] == "programming":
          #         code_lines = "\n".join(answer_data["answer"])
          #         answer = code_lines
    def get_progress_bar_chart(self,questions):
      answers_dict = self.get_progress_data(questions)
      question_titles = [q["title"] for q in questions]
      question_counts = {title: 0 for title in question_titles}
      for answered_questions in answers_dict.values():
          for q in answered_questions:
              question_counts[q] += 1

      df_counts = pd.DataFrame(list(question_counts.items()), columns=["Question", "Count"])
      plt.figure(figsize=(8,4))

      x_positions = range(len(df_counts))
        
      plt.bar(x_positions, df_counts["Count"], color="green")
        
      plt.xlabel("Questions")
      plt.ylabel("Answers submitted")
      plt.title("Answers per question")
      plt.ylim(0, len(answers_dict))
        
      plt.xticks(x_positions, df_counts["Question"], rotation=90)
      plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
        
      plt.tight_layout()

      # plt.figure(figsize=(8,4))
      # plt.bar(df_counts["Question"], df_counts["Count"], color="green")
      # plt.xlabel("Questions")
      # plt.ylabel("Answers submitted")
      # plt.title("Answers per question")
      # plt.ylim(0, len(answers_dict))
      # plt.xticks(rotation=90, ha="right")
      # plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
      # plt.tight_layout()

      return plt

    def get_progress_graph(self,questions):
      answers_dict = self.get_progress_data(questions)
      question_titles = [q["title"] for q in questions]
      df = pd.DataFrame(0, index=answers_dict.keys(), columns=question_titles)

      for user, answered_questions in answers_dict.items():
          for q in answered_questions:
              df.loc[user, q] = 1

      annot_df = df.apply(lambda col: col.map(lambda x: "✓" if x == 1 else ""))

      plt.figure(figsize=(8,8))
      sns.heatmap(df, annot=annot_df, fmt="", cmap="Greens", cbar=False, linewidths=0.5)
      plt.xlabel("Questions")
      plt.ylabel("Users")
      plt.title("User Progress")
      return plt

    def get_progress_graph2_data(self, questions):
      data = []
      for question in questions:
        correctness = question["correctness"]
        graph_data = self.get_graph_data_custom(question["title"], correctness)
        data.append(graph_data)
      return data
    
    def get_progress_graph2(self, questions):
        data = self.get_progress_graph2_data(questions)
        if not data:
            print("No data for progress graph.")
            return

        summary = []
        for i, q_data in enumerate(data):
            if not q_data:
                summary.append({"question": questions[i]["title"], "Correct": 0, "Incorrect": 0, "No Answer": 0})
                continue

            df = pd.DataFrame(q_data)
            
            df["result"] = df.apply(
                lambda row: "No Answer" if row.get("no_answer", False)
                            else ("Correct" if row.get("correct") == True else "Incorrect"),
                axis=1
            )

            counts = df["result"].value_counts()
            total = len(df)
            correct_pct = (counts.get("Correct", 0) / total) * 100
            incorrect_pct = (counts.get("Incorrect", 0) / total) * 100
            no_answer_pct = (counts.get("No Answer", 0) / total) * 100

            summary.append({
                "question": questions[i]["title"],
                "Correct": correct_pct,
                "Incorrect": incorrect_pct,
                "No Answer": no_answer_pct
            })

        summary_df = pd.DataFrame(summary)

        plt.figure(figsize=(8, 4))
        bar_width = 0.6

        plt.bar(summary_df["question"], summary_df["Correct"], color="green", label="Correct", width=bar_width)
        plt.bar(summary_df["question"], summary_df["Incorrect"], bottom=summary_df["Correct"], color="deepskyblue", label="Incorrect", width=bar_width)
        plt.bar(summary_df["question"], summary_df["No Answer"],
                bottom=summary_df["Correct"] + summary_df["Incorrect"], color="lightgray", label="No Answer", width=bar_width)

        plt.ylabel("Percentage (%)")
        plt.title("Progress Overview by Question")
        plt.xticks(rotation=90)
        plt.ylim(0, 100)
        plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
        plt.legend()
        plt.tight_layout()
        return plt

    def get_ML_DashboardProgressGraphData(self):
        data = {}
        for folder in os.listdir(self.drive_path):
            userid = folder
            data[userid] = {}
            folder_path = os.path.join(self.drive_path, folder)
            for file in os.listdir(folder_path):
                if file.endswith(".txt"):
                    file_path = os.path.join(folder_path, file)

                    with open(file_path, "r", encoding="utf-8") as f:
                        content = f.read()

                        match = re.search(r"'DataSet'\s*:\s*\(\s*'([^']+)'", content)
                        if match:
                            dataset = match.group(1)
                            if dataset not in ("auto-mpg.csv","penguins_lter.csv"):        
                                if dataset not in data[userid]:
                                    data[userid][dataset] = {"correct": 0, "incorrect": 0}
                                
                                if 'ModelPerformance' in content: 
                                    data[userid][dataset]["correct"] += 1
                                else:
                                    data[userid][dataset]["incorrect"] += 1

        return data
            
    def get_ML_DashboardProgressGraph(self,data):
        all_datasets = set()
        for user_data in data.values():
            all_datasets.update(user_data.keys())
        all_datasets = sorted(all_datasets) 
        df = pd.DataFrame(0, index=data.keys(), columns=all_datasets)

        for user, user_data in data.items():
            for dataset, scores in user_data.items():
                if scores.get("correct", 0) == 0:
                   if scores.get("incorrect", 0) > 0:
                        df.loc[user, dataset] = -1
                else:
                    df.loc[user, dataset] = 1

        annot_df = df.astype(str).mask(df == 0, "")
        annot_df = df.replace({1: "✓", -1: "✗", 0: ""})

        mask = df == 0

        plt.figure(figsize=(12, 10)) 
        sns.heatmap(
            df,
            annot=annot_df,
            fmt="",
            cmap="RdYlGn",
            center=0,
            mask=mask,
            cbar=False,
            linewidths=0.5
        )

        plt.xlabel("Datasets")
        plt.ylabel("Users")
        plt.title("Correct Answers per Dataset per User")
        plt.xticks(rotation=90, ha="center")
        ax = plt.gca()
        for label in ax.get_yticklabels():
            user = label.get_text()
            if df.loc[user].max() <= 0:
                label.set_color("red")
        plt.tight_layout()
        return plt
        

    def refresh_progress(self):
      with self.progress_graph:
        clear_output(wait=True)
        graph = self.get_progress_graph(questions)
        graph.show()

      with self.progress_bar_chart:
        clear_output(wait=True)
        graph = self.get_progress_bar_chart(questions)
        graph.show()


    def sync(self, change):
      self.download_status.value = "Downloading..."
      self.drive.get_performances_master()
      self.download_status.value = "Download complete"
      self.open_question(None)
      self.refresh_progress()

    def get_correctness(self,userid):
      current_question = currentQuiz.getCurrentQuestion()
      if current_question is None:
        return "No question selected"
      answer_file = os.path.join("drive", str(userid), current_question.getTitle() + ".json")
      if os.path.exists(answer_file):
        with open(answer_file, 'r', encoding='utf-8') as f:
            answer_data = json.load(f)
        if answer_data:
          if answer_data["type"] == "multiple_choice":
            return str(answer_data["result"])

          if answer_data["type"] == "open":
            return str(answer_data["result"])

          if answer_data["type"] == "programming":
            result = answer_data.get("result", {})
            if isinstance(result, dict):
              correct_tests = sum(v.get("correct", 0) for v in result.values())
              total_tests = len(result)
              return f"{correct_tests}/{total_tests} tests correct"

      return "No result"


      current_question = currentQuiz.getCurrentQuestion()
      if current_question.isProgrammingQuestion():
        data = self.get_graph_data_programming(self.Qqsts.value)
        if data:
            result = answer_data.get("result", {})
            if isinstance(result, dict):
                correct_tests = sum(v.get("correct", 0) for v in result.values())
                total_tests = len(result)
                return f"{correct_tests}/{total_tests}"
        else:
          return "No result"
      else:
        answer_file = os.path.join(os.path.join("drive", str(userid)), self.Qqsts.value + ".json")
        if os.path.exists(answer_file):
            with open(answer_file, 'r', encoding='utf-8') as f:
                answer_data = json.load(f)
            return str(answer_data["result"])
        return "No result"

    def show_selected_answer(self, change):
        self.answer.value = ""
        userid = change['new']
        if userid:
            ans = self.answer_mapping.get(userid, "")
            answer_index = self.get_answer_index(self.Qqsts.value,ans)

            if answer_index is None:
                answer_index = ""
            else:
              answer_index = str(answer_index) + ": "

            if isinstance(ans, dict):
                self.answer.value = ""
                for k, v in ans.items():
                    self.answer.value += f"{k}: {v}\n"
            else:
                self.answer.value = str(answer_index) + str(ans)

        self.correctness.value = str(self.get_correctness(userid))


    def getQsts(self):
        return self.Qqsts

    def getQuizTab(self):
        return self.QuizTab

    def getProgressTab(self):
      return self.ProgressTab

    def getMLDashboardTab(self):
        return self.MLDashboardTab

    ############################################################################################################################################
    def get_answers(self,question_name):
      answers = []

      folders = os.listdir(self.drive_path)
      for folder in folders:
        answer_file = os.path.join(os.path.join("drive", folder), question_name + ".json")
        if os.path.exists(answer_file):
            with open(answer_file, 'r', encoding='utf-8') as f:
                answer_data = json.load(f)
            answer = ""
            if answer_data["type"] == "multiple_choice":
                answer = answer_data["answer"]
            elif answer_data["type"] == "open":
                answer = answer_data["answer"]
            elif answer_data["type"] == "programming":
                code_lines = "\n".join(answer_data["answer"])
                answer = code_lines
            elif answer_data["type"] == "custom":
                answer = answer_data["answer"]
            answers.append((folder, answer)) #userid,answer

      return answers

    def get_answer_index(self,question_name, answer_text):
      question = next((q for q in currentQuiz.getQuestions() if q.getTitle() == question_name), None)
      if not question:
          return None

      choices = [c[0] for c in question.getChoices()]
      try:
          return choices.index(answer_text) + 1
      except ValueError:
          return None

    def check_answer_custom(self, answer, value, operator):
        try:
          #floats
            if isinstance(answer, float) or isinstance(value, float):
                tolerance = 1e-3
                if operator == "==":
                    return abs(answer - value) <= tolerance
                elif operator == "!=":
                    return abs(answer - value) >= tolerance
                elif operator == "<":
                    return answer < value - tolerance
                elif operator == "<=":
                    return answer <= value + tolerance
                elif operator == ">":
                    return answer > value + tolerance
                elif operator == ">=":
                    return answer >= value - tolerance
            else:
              #other data types
                if operator == "==":
                    return answer == value
                elif operator == "!=":
                    return answer != value
                elif operator == "<":
                    return answer < value
                elif operator == "<=":
                    return answer <= value
                elif operator == ">":
                    return answer > value
                elif operator == ">=":
                    return answer >= value
                elif operator == "in":
                    return answer in value
        except:
            return False

        return False

    def get_graph_data_custom(self,question_name, correctness=None):
      graph_data = []
      folders = os.listdir(self.drive_path)
      for folder in folders:
        answer_file = os.path.join(os.path.join("drive", folder), question_name + ".json")
        if os.path.exists(answer_file):
            with open(answer_file, 'r', encoding='utf-8') as f:
                answer_data = json.load(f)

            correct = True
            
            try:
              if answer_data.get("type") == "programming":
                    correct_keywords = answer_data.get("correct_keywords", 0)
                    total_keywords = answer_data.get("total_keywords", 0)
                    result_data = answer_data.get("result", {})

                    if result_data is None or not isinstance(result_data, dict):
                        result_data = {}

                    all_tests_passed = all(
                        v.get("correct", False)
                        for v in result_data.values()
                        if isinstance(v, dict)
                    )

                    any_test_passed = any(
                        v.get("correct", False)
                        for v in result_data.values()
                        if isinstance(v, dict)
                    )

                    if correct_keywords == total_keywords and all_tests_passed:
                        correct = True 
                    elif correct_keywords > 0 or any_test_passed:
                        correct = False
                    else:
                        correct = False  

                   
              else:
                  correctness = correctness or currentQuiz.getCurrentQuestion().correctness
                  for check in correctness:
                    key = check["key"]
                    value = check["value"]
                    operator = check["operator"]
                    answer = answer_data["answer"][key]
                    if self.check_answer_custom(answer, value,operator) == False:
                      correct = False
                      break
            except:
              correct = False

            graph_data.append({
                "correct": correct,
                "no_answer": False
            })
        else:
          #no answer
          graph_data.append({
              "correct": False,
              "no_answer": True
          })

      df = pd.DataFrame(graph_data)
      return graph_data

    def get_graph_data_mc(self,question_name):
      graph_data = []

      folders = os.listdir(self.drive_path)
      for folder in folders:
        answer_file = os.path.join(os.path.join("drive", folder), question_name + ".json")
        if os.path.exists(answer_file):
            with open(answer_file, 'r', encoding='utf-8') as f:
                answer_data = json.load(f)

            graph_data.append({
                "userid": folder,
                "answer_index": self.get_answer_index(question_name, answer_data["answer"]),
                "answer": answer_data["answer"],
                "correct": answer_data["result"],
                "no_answer": False
            })
        else:
          #no answer
          graph_data.append({
              "userid": folder,
              "answer_index": None,
              "answer": "No Answer",
              "correct": False,
              "no_answer": True
          })

      df = pd.DataFrame(graph_data)
      return graph_data



    def get_graph_data_programming(self,question_name):
      graph_data = []

      folders = os.listdir(self.drive_path)
      for folder in folders:
        answer_file = os.path.join(os.path.join("drive", folder), question_name + ".json")
        if os.path.exists(answer_file):
            with open(answer_file, 'r', encoding='utf-8') as f:
                answer_data = json.load(f)

            result = answer_data["result"]
            try:
              correct_tests = sum(v["correct"] for v in result.values()) + answer_data["correct_keywords"]
              total_tests = len(result) + answer_data["total_keywords"]
            except:
              correct_tests = 0
              total_tests = -1  


            graph_data.append({
                "userid": folder,
                "correct_tests": correct_tests,
                "total_tests": total_tests,
                "correct_keywords": answer_data["correct_keywords"],
                "total_keywords": answer_data["total_keywords"],
                "no_answer": False
            })
        else:
          #no answer
          graph_data.append({
              "userid": folder,
              "correct_tests": 0,
              "total_tests": 0,
              "correct_keywords": 0,
              "total_keywords": 0,
              "no_answer": True
          })

      return graph_data

    def plot_programming(self, question_name):
      with self.answer_bar_chart:
          clear_output(wait=True)
          graph_data = self.get_graph_data_programming(question_name)

          if not graph_data:
              print("No graph data.")
              return

          df = pd.DataFrame(graph_data)
          df["result"] = df.apply(lambda row: "No Answer" if row.get("no_answer", False) or row["total_tests"]==0
                        else ("Correct" if row["correct_tests"] == row["total_tests"] else "Incorrect"), axis=1)
          summary = df["result"].value_counts().sort_index()
          ordered_summary = summary.reindex(["Incorrect", "Correct", "No Answer"], fill_value=0)

          plt.figure(figsize=(8,4))
          plt.bar(ordered_summary.index, ordered_summary.values, color=["red","green","gray"])
          plt.xlabel("Correct tests")
          plt.ylabel("Amount")
          plt.title(f"Test results for {question_name}")

          plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
          plt.show()

    def plot_mc(self, question_name):
      with self.answer_bar_chart:
          clear_output(wait=True)
          graph_data = self.get_graph_data_mc(question_name)
          if not graph_data:
              print("No graph data.")
              return

          df = pd.DataFrame(graph_data)
          summary = df['answer_index'].value_counts()

          question = next((q for q in currentQuiz.getQuestions() if q.getTitle() == question_name), None)
          colors = ['green' if c[1] else 'red' for c in question.getChoices()] + ['gray']

          #count student answers for each option
          choices_amount = len(question.getChoices())
          counts = [0] * choices_amount + [0] #no answer
          for entry in graph_data:
            answer_idx = entry['answer_index']
            if entry['no_answer']:
                counts[-1] += 1
            if answer_idx is not None and 1 <= answer_idx <= choices_amount:
                counts[answer_idx-1] += 1

          x_axis = range(1, choices_amount+2)
          x_labels = [str(i) for i in range(1, choices_amount + 1)] + ['No Answer']

          plt.figure(figsize=(8,4))
          plt.bar(x_axis, counts, color=colors)
          plt.xticks(x_axis,x_labels)
          plt.ylabel("Amount")
          plt.xlabel("Answer choice")
          plt.title(f"Answers for {question_name}")
          plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
          plt.show()

    def plot_custom(self, Qtitle):
      with self.answer_bar_chart:
          clear_output(wait=True)
          data = self.get_graph_data_custom(Qtitle)

          if not data:
              print("No graph data.")
              return

          df = pd.DataFrame(data)
          df["result"] = df.apply(
              lambda row: "No Answer" if row.get("no_answer", False)
                          else ("Correct" if row.get("correct") == True else "Incorrect"),
              axis=1
          )

          counts = df["result"].value_counts()
          correct = counts.get("Correct", 0)
          incorrect = counts.get("Incorrect", 0)
          no_answer = counts.get("No Answer", 0)
          total = correct + incorrect + no_answer

          # Convert counts to percentages
          correct_pct = (correct / total) * 100
          incorrect_pct = (incorrect / total) * 100

          plt.figure(figsize=(6, 4))
          plt.bar(
              ["Results"], 
              [correct_pct], 
              color="green", 
              label="Correct"
          )
          plt.bar(
              ["Results"], 
              [incorrect_pct], 
              bottom=[correct_pct], 
              color="deepskyblue", 
              label="Finished"
          )

          plt.ylabel("Percentage (%)")
          plt.title(f"Results for {Qtitle}")
          plt.ylim(0, 100)
          plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
          plt.legend()
          plt.show()

    def get_keys(self):
      import json

      all_keys = set()
      folders = os.listdir(self.drive_path)
      for user in folders:
          for question in os.listdir(os.path.join("drive", user)):
              if question.endswith(".json"):
                  with open(os.path.join("drive", user, question), "r") as f:
                      data = json.load(f)
                      if "answer" in data and isinstance(data["answer"], dict):
                          all_keys.update(data["answer"].keys())

      unique_keys = list(all_keys)
      return unique_keys

    def open_question(self,b):
        self.answer.value = ""
        self.answers.value = None
        global currentQuiz,BOLD,RESET
        for qstn in currentQuiz.getQuestions():
            if qstn.getTitle() == self.Qqsts.value:
                currentQuiz.setCurrentQuestion(qstn)
                break

        if currentQuiz.getCurrentQuestion() is None:
           return

        try:
            Qtitle = currentQuiz.getCurrentQuestion().getTitle()
            QText = currentQuiz.getCurrentQuestion().getText()

            with self.description_out:
                clear_output(wait=True)
                print(f"""{BOLD}{Qtitle}{RESET}""")
                print("_______________________________")
                print(QText)
                print()
                if currentQuiz.getCurrentQuestion().IsMChoice():
                    self.answers.options = [x[0] for x in currentQuiz.getCurrentQuestion().getChoices()]
                    for i in range(len(currentQuiz.getCurrentQuestion().getChoices())):
                        print(f"{i+1}. {currentQuiz.getCurrentQuestion().getChoices()[i][0]}")

                answers = self.get_answers(Qtitle)

                if answers:
                    self.answers.options = [userid for userid, _ in answers]
                    self.answer_mapping = {userid: ans for userid, ans in answers}
                else:
                    self.answers.options = []
                    self.answer_mapping = {}

            if currentQuiz.getCurrentQuestion().isProgrammingQuestion():
              self.plot_programming(Qtitle)
            elif currentQuiz.getCurrentQuestion().IsMChoice():
              self.plot_mc(Qtitle)
            elif currentQuiz.getCurrentQuestion().isCustomQuestion():
              # self.keys.options = self.get_keys()
              self.plot_custom(Qtitle)
            else:
              with self.answer_bar_chart:
                clear_output(wait=True)

        except Exception as e:
            self.answer.value = 'open quest error .. '+str(e)

        return
    ##############################################################################################################



def open_quiz(VisManager,tab_set, questions):

    global currentQuiz

    qtslist = []

    quizstr = "Quiz Deel 1_Questions"

    tab_set.set_title(0,'Quiz')
    tab_set.set_title(1,'Progress overview')
    tab_set.set_title(2, 'ML dashboard stats')

    currentQuiz = Quiz('Quizs')

    #rel_path = 'Quizzes'+'\\'+quizstr+'.csv'
    #abs_file_path = os.path.join(Path.cwd(), rel_path)

    for i in range(len(questions)):
        newquestion = Question(questions[i]["title"],questions[i]["text"],questions[i]["type"], questions[i]["correctness"])
        # newquestion = Question(r['Title'],r['Text'],r['Correctness'].find('~~')>-1)
        correctness = questions[i]["correctness"]
        if questions[i]["type"] != "custom":
          choices = questions[i]["choices"]
          if newquestion.IsMChoice():
              for i in range(len(choices)):
                  newquestion.getChoices().append((choices[i],correctness[i]))
          else:
              newquestion.getChoices().append((questions[i]["choices"],questions[i]["correctness"]))

        currentQuiz.getQuestions().append(newquestion)

    VisManager.getQsts().options = [x.getTitle() for x in currentQuiz.getQuestions()]



    return
##################################################################################################################################################

#######################

questionlist = QuestionsList()

questions = questionlist.get_questions_master_version()

VisMgr = VisualManager(questions)
tab_set = widgets.Tab([VisMgr.getQuizTab(),VisMgr.getProgressTab(), VisMgr.getMLDashboardTab()])
open_quiz(VisMgr,tab_set, questions)
tab_set
